In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
import models

In [58]:
import os
os.makedirs("./saved_models/", exist_ok=True)

In [4]:
train_VAE = True
train_GAN = False

In [5]:
E = 200 # GeV
M = 114 # GeV
c = 1

In [6]:
# input_dimension = 50
# data = np.random.beta(1, 2, (100_000, input_dimension))

In [7]:
data = np.loadtxt("data/events.txt")

# Delete Energy (E) entries
# data = np.delete(data, [0, 4], axis=1)

# Delete theta and phi
data = np.delete(data, [8, 9], axis=1)
input_dimension = data.shape[1]

In [8]:
data.mean(), data.std()

(25.24994401635484, 66.4280034159734)

In [9]:
data.shape

(100000, 8)

In [10]:
latent_dim = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'
vae = models.VAE(input_dimension, latent_dim, device=device).to(device)
vae_optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3) 


In [11]:
print(device, vae_optimizer.defaults)

cpu {'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}


In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(vae)

1365208

In [13]:
class PCDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        if self.transform:
            item = self.transform(item)
        return item

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test = train_test_split(data, test_size=0.15, random_state=42)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
print(X_train.mean(), X_train.std())
print(X_test.mean(), X_test.std())

4.988028616292231e-12 0.9999999999999999
2.5556058295948083e-05 1.0032579828462234


In [16]:
train_dataset = PCDataset(X_train)
val_dataset = PCDataset(X_test)

In [17]:
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=True)

In [18]:
import time

In [19]:
def train_VAE_helper(epochs):
    vae.train()

    for i in range(epochs):

        start_time = time.time()

        vae_loss_avg, recon_loss_avg, KL_loss_avg = 0, 0, 0

        for batch in train_dataloader:

            vae_loss, recon_loss, KL_loss = vae.loss_function(batch.float().to(device))

            vae_loss_avg += vae_loss.detach().cpu().numpy() / len(train_dataloader)
            recon_loss_avg += recon_loss.detach().cpu().numpy() / len(train_dataloader)
            KL_loss_avg += KL_loss.detach().cpu().numpy() / len(train_dataloader)

            vae_optimizer.zero_grad()
            vae_loss.backward()
            vae_optimizer.step()

        print("Epoch:", i, f"time passed: {time.time() - start_time:.2f}s", f"Train Losses (avg.): {vae_loss_avg:.5f}, {recon_loss_avg:.5f}, {KL_loss_avg:.5f}")

In [20]:
if train_VAE:
    train_VAE_helper(25)
    torch.save(vae.state_dict(), "./saved_models/vae-25e")

Epoch: 0 time passed: 12.153722524642944s Train Losses (avg.): 0.14633, 0.06960, 0.15345
Epoch: 1 time passed: 12.804848194122314s Train Losses (avg.): 0.07340, 0.02026, 0.10628
Epoch: 2 time passed: 13.565009832382202s Train Losses (avg.): 0.06923, 0.01767, 0.10311
Epoch: 3 time passed: 13.57044243812561s Train Losses (avg.): 0.06812, 0.01709, 0.10207
Epoch: 4 time passed: 13.957868814468384s Train Losses (avg.): 0.06749, 0.01680, 0.10138


In [21]:
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7
count,8.500000e+04,8.500000e+04,8.500000e+04,8.500000e+04,8.500000e+04,8.500000e+04,8.500000e+04,8.500000e+04
mean,4.118267e-11,2.825452e-17,-4.965440e-17,-7.638784e-15,-1.278439e-12,-2.825452e-17,4.965440e-17,7.638784e-15
std,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00
min,-4.049064e+00,-1.946276e+00,-1.952758e+00,-1.065234e+00,-4.343428e+00,-1.946913e+00,-1.936472e+00,-2.781868e+00
25%,-6.769501e-01,-6.963382e-01,-7.025040e-01,-9.202029e-01,-6.740192e-01,-6.998756e-01,-6.994429e-01,-8.392746e-01
50%,1.601171e-03,8.194046e-04,-2.646193e-03,-3.354269e-01,1.623124e-03,-8.194046e-04,2.646193e-03,3.354269e-01
75%,6.771499e-01,6.998756e-01,6.994429e-01,8.392746e-01,6.755138e-01,6.963382e-01,7.025040e-01,9.202029e-01
max,4.266014e+00,1.946913e+00,1.936472e+00,2.781868e+00,4.197578e+00,1.946276e+00,1.952758e+00,1.065234e+00


In [22]:
input = torch.tensor(X_test).float().to(device)

In [23]:
input

tensor([[-0.4450,  1.7686, -0.5562,  ..., -1.7686,  0.5562, -0.4366],
        [ 1.0305,  0.5344, -1.8574,  ..., -0.5344,  1.8574, -1.4185],
        [ 0.9248,  1.8641,  0.5211,  ..., -1.8641, -0.5211, -1.3171],
        ...,
        [ 0.7533, -1.4933, -1.2526,  ...,  1.4933,  1.2526, -1.0162],
        [-1.6089, -0.9841,  1.6691,  ...,  0.9841, -1.6691, -1.1514],
        [-0.7212, -0.6345,  1.7932,  ...,  0.6345, -1.7932, -1.5185]])

In [24]:
vae.eval()
output, _, _ = vae(input.float())

In [25]:
output.detach().cpu().numpy()

array([[-0.6619132 ,  1.8554926 , -0.488048  , ..., -1.8328332 ,
         0.49037448, -0.37101728],
       [ 0.6717782 ,  0.56252086, -1.9046621 , ..., -0.5728383 ,
         1.9106606 , -1.3387259 ],
       [ 0.87879384,  1.9015018 ,  0.5199303 , ..., -1.8876109 ,
        -0.5116108 , -1.3788307 ],
       ...,
       [ 0.84217685, -1.2721606 , -1.0857284 , ...,  1.2733922 ,
         1.112038  , -1.0509855 ],
       [-2.1938047 , -0.68161976,  1.5676273 , ...,  0.6738301 ,
        -1.5943291 , -0.97613406],
       [-0.8350672 , -0.5445279 ,  1.6054653 , ...,  0.5095526 ,
        -1.6048576 , -1.5104892 ]], dtype=float32)

In [26]:
import torch.nn.functional as F
F.mse_loss(input, output)

tensor(0.0211, grad_fn=<MseLossBackward0>)

In [27]:
scaler.inverse_transform(output.detach())

array([[100.9668623 ,  95.30042671, -24.67862163, ..., -94.13640288,
         24.79825414,  33.91245792],
       [101.03332568,  28.87993382, -97.52363872, ..., -29.40994523,
         97.8320927 , -10.61561369],
       [101.04364213,  97.66393748,  27.15355584, ..., -96.95035812,
        -26.72575134, -12.46099044],
       ...,
       [101.04181735, -65.3684147 , -55.41249545, ...,  65.43168   ,
         56.76538604,   2.62445422],
       [100.89052179, -35.03208499,  81.02814596, ...,  34.63192635,
        -82.40120416,   6.06866052],
       [100.95823332, -27.98961863,  82.9738476 , ...,  26.19292349,
        -82.94259701, -18.51911707]])

In [28]:
scaler.inverse_transform(input.detach())

array([[100.97767136,  90.83617262, -28.18133728, ..., -90.83617262,
         28.18133728,  30.89663198],
       [101.05120271,  27.43679191, -95.09550229, ..., -27.43679191,
         95.09550229, -14.28524883],
       [101.04593388,  95.7434981 ,  27.2143253 , ..., -95.7434981 ,
        -27.2143253 ,  -9.62098303],
       ...,
       [101.03738629, -76.72693233, -63.99310538, ...,  76.72693233,
         63.99310538,   4.22612515],
       [100.91967194, -50.57134785,  86.24701884, ...,  50.57134785,
        -86.24701884,  -1.997712  ],
       [100.96390777, -32.61309232,  92.62601312, ...,  32.61309232,
        -92.62601312, -18.88935201]])

In [29]:
gan = models.GAN(input_shape = input_dimension)

In [30]:
count_parameters(gan), count_parameters(gan.discriminator), count_parameters(gan.generator)

(850825, 136193, 714632)

In [31]:
def train_GAN_helper(epochs):
    for i in range(epochs):

        start_time = time.time()

        d_loss_avg, g_loss_avg = 0, 0

        for batch in train_dataloader:

            d_loss, g_loss = gan.train_with_batch(batch.float().to(device))

            d_loss_avg += d_loss / len(train_dataloader)
            g_loss_avg += g_loss / len(train_dataloader)

        print("Epoch:", i, f"time passed: {time.time() - start_time:.2f}s", f"Train Losses (avg.): {d_loss_avg:.5f}, {g_loss_avg:.5f}")

In [32]:
if train_GAN:
    train_GAN_helper(25)
    torch.save(gan.state_dict(), "./saved_models/gan-25e")

In [51]:
gan.eval()
gan_generated = scaler.inverse_transform(gan.generate(15_000).detach())

In [52]:
pd.DataFrame(gan_generated).describe()

,0,1,2,3,4,5,6,7
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,101.001361,-1.222727,1.676517,-51.858985,101.000300,-1.231672,-1.760722,50.251142
std,0.000253,0.260430,0.229520,0.217810,0.000262,0.279305,0.295966,0.251015
min,101.000367,-2.278667,0.819331,-52.788914,100.999218,-2.254470,-3.163085,49.248861
25%,101.001194,-1.396561,1.522865,-52.000284,101.000126,-1.414858,-1.955615,50.081825
50%,101.001365,-1.224129,1.675602,-51.859144,101.000302,-1.232864,-1.756632,50.246094
75%,101.001532,-1.047620,1.824654,-51.714294,101.000473,-1.042273,-1.561415,50.414925
max,101.002287,-0.195210,2.612123,-51.059563,101.001478,-0.187642,-0.481164,51.255618


In [56]:
dummy_data = torch.Tensor(torch.normal(0, 1, size=(15_000, latent_dim)))
vae.eval()
vae_generated = scaler.inverse_transform(vae.decode(dummy_data).detach())

In [57]:
pd.DataFrame(vae_generated).describe()

,0,1,2,3,4,5,6,7
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,100.997593,1.991920,0.253128,-51.775621,101.001311,-2.183792,0.092665,52.145582
std,0.048513,45.561888,46.302918,39.880162,0.048363,45.505342,46.295134,39.899395
min,100.798654,-172.643933,-169.316013,-117.510798,100.833997,-162.445436,-178.031218,-129.275435
25%,100.964660,-27.170694,-30.213697,-85.758905,100.968583,-31.286243,-29.564856,24.705323
50%,100.997560,1.885618,-1.295408,-60.151503,101.000734,-2.025924,1.472877,60.591064
75%,101.030792,31.143385,29.962278,-24.240006,101.033413,27.032923,30.748242,86.153789
max,101.182409,161.773554,178.150900,130.272144,101.218560,170.392102,169.480615,116.943943


In [44]:
pd.DataFrame(scaler.inverse_transform(X_test)).describe()

,0,1,2,3,4,5,6,7
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,100.999625,-0.251671,-0.124810,-50.612889,100.999935,0.251671,0.124810,50.612889
std,0.049860,51.369773,51.607820,46.222633,0.050101,51.369773,51.607820,46.222633
min,100.805305,-99.954406,-99.988495,-100.000000,100.806001,-99.980538,-99.999600,-70.612768
25%,100.965652,-37.010077,-36.297777,-93.403606,100.966527,-35.451142,-35.394070,11.379495
50%,100.999480,0.017226,-0.051379,-65.606204,100.999942,-0.017226,0.051379,65.606204
75%,101.033186,35.451142,35.394070,-11.379495,101.033859,37.010077,36.297777,93.403606
max,101.184850,99.980538,99.999600,70.612768,101.188842,99.954406,99.988495,100.000000


In [45]:
pd.DataFrame(scaler.inverse_transform(np.random.normal(0, 1, size=(15_000, 8)))).describe()

,0,1,2,3,4,5,6,7
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,100.999909,0.362520,0.405919,-51.034219,100.998868,0.123345,-1.013253,51.739667
std,0.049412,51.527085,51.703501,46.013523,0.049497,51.242061,51.114680,46.066964
min,100.792828,-202.413530,-214.717237,-239.391667,100.747439,-203.018754,-203.479676,-115.369946
25%,100.966854,-35.015952,-34.077487,-82.535428,100.965731,-34.262735,-35.679834,20.873839
50%,101.000000,0.550670,0.381733,-51.384506,100.998627,0.127280,-0.589876,51.485166
75%,101.032855,35.671081,35.516530,-19.021502,101.031660,34.628668,33.614885,82.737974
max,101.197089,185.797196,194.943398,122.900389,101.174115,182.342540,179.812768,257.444652
